<a href="https://colab.research.google.com/github/jp3256/blink_detector/blob/main/blink_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# connect to google drive folder
from google.colab import drive
import glob
drive.mount('/content/drive')
file_directory1 = '/content/drive/My Drive/Applied Research/data/final_study/batch1/'
file_directory2 = '/content/drive/My Drive/Applied Research/data/final_study/batch2/'
file_directory3 = '/content/drive/My Drive/Applied Research/data/final_study/batch3/'
landmark_predictor_directory = '/content/drive/My Drive/Applied Research/data/shape_predictor_68_face_landmarks.dat'

# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils import face_utils
import numpy as np
import pandas as pd
import imutils
import time
import dlib
import cv2
import re
from google.colab.patches import cv2_imshow

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# function to calculate eye aspect ratio
def eye_aspect_ratio(eye):
    # vertical distance between eye landmark coordinates
    vertical1 = dist.euclidean(eye[1], eye[5])
    vertical2 = dist.euclidean(eye[2], eye[4])

    # horizontal distance between eye landmark coordinates
    horizontal = dist.euclidean(eye[0], eye[3])

    # compute the eye aspect ratio
    eye_aspect_ratio = (vertical1 + vertical2) / (2.0 * horizontal)

    return eye_aspect_ratio

In [3]:
# function to get number of blinks
# code reference: https://www.pyimagesearch.com/2017/04/24/eye-blink-detection-opencv-python-dlib/
def get_num_blinks(video_path):
    # start video stream thread
    vs = FileVideoStream(video_path).start()
    time.sleep(1.0)
    stop = False
    frame_count = 0
    total_blinks = 0
    # loop over frames
    while stop==False:
        # check if there are more frames to process
        if vs.more():
            # get frame from video file stream
            frame = vs.read()
            if frame is not None:
                # resize frame
                frame = imutils.resize(frame, width=450)
                # convert to grayscale
                grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                # detect face
                face_rectangle = face_detector(grayframe, 0)
                for rect in face_rectangle:
                    # detect facial landmarks
                    shape = landmark_predictor(grayframe, rect)
                    # convert facial landmark (x, y)-coordinates to numpy array
                    shape = face_utils.shape_to_np(shape)
                    # get left and right eye coordinates
                    left_eye = shape[l_start:l_end]
                    right_eye = shape[r_start:r_end]
                    # calculate EAR for both eyes and average them
                    left_ear = eye_aspect_ratio(left_eye)
                    right_ear = eye_aspect_ratio(right_eye)
                    average_ear = (left_ear + right_ear) / 2.0
                    # draw contours around each eye by computing the convex hull 
                    left_eye_hull = cv2.convexHull(left_eye)
                    right_eye_hull = cv2.convexHull(right_eye)
                    cv2.drawContours(frame, [left_eye_hull], -1, (255, 0, 0), 1)
                    cv2.drawContours(frame, [right_eye_hull], -1, (255, 0, 0), 1)
                    # if EAR < blink threshold, increment frame count
                    if average_ear < EAR_THRESHOLD:
                        frame_count += 1
                    else:
                        # if eyes were closed for NUM_CONSEC_FRAMES, increment total blinks
                        if frame_count >= NUM_CONSEC_FRAMES:
                            total_blinks += 1
                        # reset frame counter
                        frame_count = 0
                    # # display total number of blinks and EAR on the frame
                    # cv2.putText(frame, "# Blinks: {}".format(total_blinks), (10, 30),
                    #             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    # cv2.putText(frame, "EAR: {:.2f}".format(average_ear), (300, 30),
                    #             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                stop = True
            # # show the frame
            # cv2_imshow(frame)
    # end
    cv2.destroyAllWindows()
    vs.stop()
    return total_blinks

In [4]:
# define hyperparameters (EAR threshold, # consecutive frames < EAR threshold)
EAR_THRESHOLD = 0.25
NUM_CONSEC_FRAMES = 3

# initialize dlib face detector (HOG-based) & facial landmark predictor (pre-trained)
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor(landmark_predictor_directory)

# get indexes of facial landmarks for left and right eye
(l_start, l_end) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(r_start, r_end) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [ ]:
file_directories = [file_directory1, file_directory2, file_directory3]
# loop through batches
for file_directory in file_directories:
    print(file_directory)
    # compile results
    blink_results = []
    for file in glob.glob(file_directory+'*.mp4'):
        group_number = re.search(r'(?<=batch)([0-9]+)/([0-9]+)', file).group(2)
        condition = re.search(r'(?<=batch)([0-9]+)/([0-9]+)(-*)([A-Za-z]+)', file).group(4).lower()
        role = re.search(r'(?<=batch)([0-9]+)/([0-9]+)(-*)([A-Za-z]+)(-*)([A-Za-z]+)(-*)([A-Za-z]+)*', file).group(8)
        number_blinks = get_num_blinks(file)
        blink_results.append({'group_number': group_number,
                        'condition': condition,
                        'role': role,
                        'number_blinks': number_blinks})
        print(group_number)
    df = pd.DataFrame(blink_results)
    df.to_csv(file_directory + 'batch_results_ear_0_25_3frames.csv', index=False)